In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import time

In [5]:
LINKS = []
#Number of pages we want to scrape
PAGES = 2    

# Itterating over the pages
for PAGE in range(1, PAGES + 1):
    TARGET_URL = f'https://www.avito.ma/fr/maroc/voitures?o={PAGE}'
    HEADERS = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"}
    RESPONSE = requests.get(TARGET_URL, headers=HEADERS)
    PAGE_CONTENT = bs(RESPONSE.content, "html.parser")
    LISTINGS = PAGE_CONTENT.find_all("a", class_="sc-1jge648-0")
    # Getting the link of each listing and append it to out listings list
    for LISTING in LISTINGS:
        LINK = LISTING.get("href") if LISTING and LISTING.get("href") else "N/A"
        LINKS = np.append(LINKS, LINK)

In [6]:
# Dropping duplicat links
link_sr = pd.Series(LINKS)
link_sr = link_sr.drop_duplicates().reset_index(drop=True)
link_sr.value_counts()
# Array of links
links = np.array(link_sr) 

In [7]:
listings = []
headers = {"user-agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")}
#Itterating over the links we got
page_num = 0
for link in links:
    page_num += 1
    start = time.perf_counter()
    response = requests.get(link, headers=headers, timeout=10)
    page = bs(response.content, "html.parser")

    # Link
    listing = {"lien": link}

    # Listing title
    title = page.find("h1", class_="sc-16573058-5 izVEJU")
    listing["titre annonce"] = title.text.strip() if title else None

    # Price
    price = page.find("div", class_="sc-16573058-10 kRLGQQ")
    listing["price"] = price.text.strip() if price else None

    # Address & date
    spans = page.find_all("span", class_="sc-16573058-17 gLkxLA")
    listing["spans"] = [span.text.strip() for span in spans] if spans else None

    # Owner
    owner = page.find("p",class_="sc-1x0vz2r-0 fUTtTl sc-1l0do2b-9 bJuYLD")
    listing["proprietere"] = owner.text.strip() if owner else None

    # Tags
    spans = page.find_all("span", class_="sc-1x0vz2r-0 fjZBup")
    listing["tags"] = [span.text.strip() for span in spans] if spans else None
    # Images
    listing["images"] = [img.get("src") for img in page.find_all("img", class_="sc-1gjavk-0 fpXQoT") if img.get("src")] if page.find_all("img", class_="sc-1gjavk-0 fpXQoT") else None
    
    # Append listing to listings array
    listings.append(listing)

    # Waiting a bit so we dont get banned
    elapsed = time.perf_counter() - start
    print(f"page {page_num} scraped in {elapsed:.2f}s")
    time.sleep(1.5)

page 1 scraped in 0.80s
page 2 scraped in 0.76s
page 3 scraped in 0.71s
page 4 scraped in 1.41s
page 5 scraped in 0.80s
page 6 scraped in 0.85s
page 7 scraped in 0.88s
page 8 scraped in 0.89s
page 9 scraped in 0.87s
page 10 scraped in 0.94s
page 11 scraped in 0.90s
page 12 scraped in 0.68s
page 13 scraped in 0.86s
page 14 scraped in 1.16s
page 15 scraped in 0.73s
page 16 scraped in 0.75s
page 17 scraped in 0.76s
page 18 scraped in 0.81s
page 19 scraped in 1.34s
page 20 scraped in 0.74s
page 21 scraped in 0.78s
page 22 scraped in 0.69s
page 23 scraped in 0.73s
page 24 scraped in 0.87s
page 25 scraped in 0.87s
page 26 scraped in 0.89s
page 27 scraped in 0.89s
page 28 scraped in 0.68s
page 29 scraped in 0.71s
page 30 scraped in 0.70s
page 31 scraped in 0.76s
page 32 scraped in 0.94s
page 33 scraped in 0.84s
page 34 scraped in 0.92s
page 35 scraped in 0.92s
page 36 scraped in 0.70s
page 37 scraped in 0.78s
page 38 scraped in 0.80s
page 39 scraped in 0.70s
page 40 scraped in 0.75s
page 41 s

In [10]:
df_raw = pd.DataFrame(listings)
df_clean = df_raw.copy()

df_clean.head()


,lien,titre annonce,price,spans,proprietere,tags,images
0,https://www.avito.ma/fr/hay_el_qods/voitures_d...,Mercedes-Benz GLA 220 d 4MATIC – 2026,,"[Hay El Qods, Oujda, il y a 15 heures, Hay El ...",C ZAM AUTO oujda,"[Voitures d'occasion, à vendre, 2025, Automati...",[https://content.avito.ma/classifieds/images/1...
1,https://www.avito.ma/fr/2_mars/voitures_d_occa...,Mercedes g 63 Amg,,"[2 Mars, Casablanca, il y a 23 heures, 2 Mars,...",RT auto,"[Voitures d'occasion, à vendre, 2022, Automati...",[https://content.avito.ma/classifieds/images/1...
2,https://www.avito.ma/fr/almaz/voitures_d_occas...,Audi Q3 Diesel Automatique 2021 à Casablanca,,"[Almaz, Casablanca, il y a 2 heures, Almaz, Ca...",Auto Aymane,"[Voitures d'occasion, à vendre, 2016, Automati...",[https://content.avito.ma/classifieds/images/1...
3,https://www.avito.ma/fr/maarif/voitures_d_occa...,Range Rover Sport HSE,Prix à la baisse480 000 DH,"[Maarif, Casablanca, il y a 8 minutes, Maarif,...",AUTO REDA,"[Voitures d'occasion, à vendre, 2020, Automati...",[https://content.avito.ma/classifieds/images/1...
4,https://www.avito.ma/fr/maarif/voitures_d_occa...,Porsche Cayenne Coupé Sport Design,,"[Maarif, Casablanca, il y a 6 minutes, Maarif,...",AUTO REDA,"[Voitures d'occasion, à vendre, 2024, Automati...",[https://content.avito.ma/classifieds/images/1...
